We define the intensity error $e_I$ as in:

$$ e_I = I^{*}(x) - I(W(x,p))$$

Where $x$ are 2d positions on the image plane and $W(x,p)$ is the warping function that warps pixels from the image $I$ to the template $I*$.

The warp depends on the camera parameters, the depth at $x$ and the relative pose. As we are using calibrated RGBD cameras, all parameters are given a part from the relative pose which is what we are trying to find. The warping function is defined as follows:

$$ W(x,p) = \pi(g(\pi^{-1}(x),p))$$

Where g(x,p) is an SE3 transformation consisting of a rotation and translation of a rigid body.

Since the image and the warp are non linear functions we perform a first order taylor expansion and solve the approximated error iteratively for a parameter update $\Delta p$. 

$$  e_I \approx I^{*}(W(x,0)) + \nabla I^{*} \nabla \pi \frac{\delta g}{\delta p} \Delta p - I(W(x,p)) $$

Note how we formulate the update to the warp in terms a warp of the template to the image, while the actual warp is formulated in terms of image to template. This is called the inverse compositional approach and we will see in a bit how it allows us to precompute the jacobians. However, with this formulate we have the update the parameters "inversely":

$$ W(x,p_1) = \pi(g(\pi^{-1}(\pi(g(\pi^{-1}(x),p_0))),\Delta p)) $$

$$ W(x,p_1) = \pi(g(g(\pi^{-1}(x),p_0),-\Delta p)) $$

$$ g(x,p_1) = g(g(x,p_0),-\Delta p) $$

$$ W(x,p_1) = \pi(g(\pi^{-1}(x),p_1)) $$



As $g(x,p)$ is a SE3 transformation its derivative is not trivial. However, since we are only interested in an small increment around $p$ we can formulate it as an increment in the lie algebra:

$$ g = e^{\hat{\Delta p}} \boxplus g(x,p = 0)$$

Where $e$ is the matrix exponential, $\hat{\Delta p}$ is the skew symmetric matrix of $\Delta p$ and $\boxplus$ is the group operator of SE3. We can compute the jacobian of the expression following "A tutorial on SE(3) transformation parameterizations and
on-manifold optimization" appendix A.2 "Applications to Computer Vision".

A transformation of a point p can be expressed as:

$$  e^{\hat{\Delta p}} \boxplus g(x,p) \boxplus p $$

To compute the derivative we can compute $g(x,p) \boxplus p$ which is simply the transformed point $p^{+}$

$$  e^{\hat{\Delta p}} \boxplus p^{+} $$

Which is #TODO why?

$$  \hat{\omega} p^{+} + t$$

With $\hat{\omega }$ being the skew symmetric matrix of the rotational part of $\Delta p$ and $t$ being the translational part:

$$ \begin{bmatrix} 
	0 & -\omega_3 & \omega_2 \\
	\omega_3 & 0 & -\omega_1\\
	-\omega_2 & \omega_1 & 0 \\
	\end{bmatrix} p^{+} + t$$

$$ \begin{bmatrix} 
	0 & -\omega_3 & \omega_2 \\
	\omega_3 & 0 & -\omega_1\\
	-\omega_2 & \omega_1 & 0 \\
	\end{bmatrix} \begin{bmatrix} p_x \\ p_y\\ p_z \end{bmatrix} + \begin{bmatrix} t_x \\ t_y\\ t_z \end{bmatrix}$$

$$ fx = -\omega_3 p_y + \omega_2 p_z + t_x $$
$$ fy = \omega_3 p_x - \omega_1 p_z + t_y $$
$$ fz = -\omega_2 p_x + \omega_1 p_y + t_z $$

$$ \frac{\delta fz}{\delta \omega_1} = p_y $$
$$ \frac{\delta fz}{\delta \omega_2} = -p_x $$
$$ \frac{\delta fz}{\delta \omega_3} = 0 $$
$$ \frac{\delta fz}{\delta t_x} = 0 $$
$$ \frac{\delta fz}{\delta t_y} = 0 $$
$$ \frac{\delta fz}{\delta t_z} = 1.0 $$



#TODO show full Jacobian



Additionally, we can parameterize the warp in such a way that W(x,0) is the identity warp. This yields:

$$  e_I \approx I^{*}(x)- I(W(x,p)) + \nabla I^{*} \nabla \pi \frac{\delta g}{\delta p} \Delta p $$
$$  e_I \approx I^{*}(x)- I(W(x,p)) + J_I J_{\pi} J_T \Delta p $$



#TODO use matrix formulation, include weights

Where we can summarize to:

$$  e_I \approx r + J\Delta p $$

$$  e_I \approx r^Tr + 2J^Tr\Delta p + (J\Delta p)^TJ\Delta p $$

Since we are interested in the minimum of e_I we set its derivative to 0 and solve for $\Delta p$:

$$ \frac{\delta e_I}{\delta p} = 2J^Tr + 2J^TJ \Delta p = 0$$

$$ \Delta p = - \frac{J^Tr}{J^TJ}


Since we are in rgbd setting we can impose an additional constraint on the depth:

$$ e_Z = [h(Z^{*}(x),p)]_z - Z(W(x,p))$$

Where $h$ is the transformation from the pixel at $Z^{*}$ to the camera coordinate system of $Z$ and $[.]_z$ selects the z-component of the vector.

$$ h = g(\pi^{-1}(x),p) $$

Where  $\pi()$ is the camera projection and g(x,p) is an SE3 transformation. Note how it is simply the warping function without the final reprojection part:

$$ W(x,p) = \pi(g(\pi^{-1}(x),p))$$

Similarly to the intensity error we can express the depth error in terms of a small increment. However, in case of the depth we have the relative pose not only in the warping function but also in the transformation function $h$:

$$ e_Z = Z^{*}(W(x,\Delta p)) - [g(g^{-1}(\pi(W(x,p)),p)\Delta p)]_z$$

And write the whole expression as:

$$ e_Z = Z^{*}(\pi(g(\pi^{-1} (x), \Delta p))) - [g(g^{-1}(Z(\pi(W(x,p))),p)\Delta p)]_z$$

We can linearize the error for a small increment $\Delta p$:

$$ e_Z \approx e_Z(0) + \frac{\delta e_Z}{\delta p}\Delta p = Z^{*}(x) - [g^{-1}(\pi^{-1}(x)Z(W(x,p)),p)]_z + (\nabla Z^{*}\nabla \pi\frac{\delta g}{\delta p} - [\frac{\delta g}{\delta p})]_z\Delta p$$

Note that in the first expression of the derivative we compute $\frac{\delta g}{\delta p}$ at identity $p=0$ and the expression is constant. In the second part we compute it at the warped position $p=p + \Delta p$, so we have recompute it each iteration.


We can summarize to:

$$ e_Z \approx (r_z + J_z) $$

Where:

$$ r_z = Z^{*}(x) - [g^{-1}(\pi^{-1}(x)(Z(W(x,p))),p)]_z $$

is the difference between the depth of of $Z^{*}$ and the depth at the warped pixel position $W(x,p)$ in $Z$ transformed back to the coordinate system of $Z^{*}$

And:

$$ J_z = J_Z J_{\pi} J_T + [J_T]_Z $$

Where $J_Z$ is the x and y derivative of $Z^{*}$, $J_{\pi}$ is the gradient of the image projection with respect to x,y and z and $J_T$ is the derivative of an SE3 transformation with respect to a small increment in the lie algebra $x,y,z,rx,ry,rz$.

and will reach similar normal equations as for e_I.
 

If we combine the intensity and depth constraints for each observation we get:

$$e_I + we_Z \approx r_Z + wr_I + ((J_IJ_{\pi}J_T)+w(J_ZJ_{\pi}J_T+[J_{T}]_z))\Delta p$$

Where w can be a weighting factor to balance the scales of the two errors.